In [1]:
from typing import List

import pickle
import os
import time
from math import dist
import random
import json
import sys
from ast import literal_eval
import copy
import traceback


import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils.gamehelper import PoeBot, Entity
from utils.controller import VMHostPuppeteer
from utils.utils import cropPath, sortByHSV, alwaysFalseFunction, raiseLongSleepException, createLineIterator, createLineIteratorWithValues, getFourPoints, extendLine
from utils.helper_functions import openStashTabIndexKeryboard, checkIfInventoryOpened, getFilledInventorySlots, openStash, getToHideout, getToPortal, mapDiscoveredForPercent, lvlUpGem, openTradeWithLilly
from utils.constants import FRIENDLY_ENTITIES_PATH_KEYWORDS, MAP_DEVICE_SLOTS, MAP_DEVICE_ACTIVATE_BUTTON, FLASK_NAME_TO_BUFF, BOSS_RENDER_NAMES
from utils.temps import MapsTempData
from utils.pathing import TSP
from utils.mover import Mover


In [2]:
time_now = 0
notebook_dev = False
# readability
poe_bot: PoeBot
bot_controls:VMHostPuppeteer
mover: Mover

In [3]:
default_config = {
  "REMOTE_IP": '172.26.26.246', # z2
  "UNIQUE_ID": "price_checker_test",
  "force_reset_temp": False,
  "password": None,
}



try:
  i = sys.argv[1]
  print(i)
  parsed_config = literal_eval(i)
  print(f'successfully parsed cli config')
  print(f'parsed_config: {parsed_config}')
except:
  print(f'cannot parse config from cli, using default\dev one')
  notebook_dev = True
  parsed_config = default_config

config = {

}

for key in default_config.keys():
  try:
    config[key] = parsed_config[key]
  except:
    config[key] = default_config[key]

print(f'config to run {config}')

--f=c:\Users\1\AppData\Roaming\jupyter\runtime\kernel-v2-16140uLvHq2bb2Mbw.json
cannot parse config from cli, using default\dev one
config to run {'REMOTE_IP': '172.26.26.246', 'UNIQUE_ID': 'price_checker_test', 'force_reset_temp': False, 'password': None}


In [4]:
REMOTE_IP = config['REMOTE_IP'] # REMOTE_IP
UNIQUE_ID = "price_checker_test" # unique id
force_reset_temp = config['force_reset_temp']
mule_give_category = ["deafing_essences", "gilded_scarabs"]
print(f'running mule_giver_test using: REMOTE_IP: {REMOTE_IP} unique_id: {UNIQUE_ID} force_reset_temp: {force_reset_temp}')

running mule_giver_test using: REMOTE_IP: 172.26.26.246 unique_id: price_checker_test force_reset_temp: False


In [5]:
poe_bot = PoeBot(unique_id=UNIQUE_ID, remote_ip = REMOTE_IP)
bot_controls = poe_bot.bot_controls
stash = poe_bot.ui.stash
inventory = poe_bot.ui.inventory
coordinator = poe_bot.coordinator



trying to load StashTempData uid: price_checker_test
loaded StashTempData uid: price_checker_test
trying to load AfkTempData uid: price_checker_test
loaded AfkTempData uid: price_checker_test
CombatModule.build is not assigned, using any functions may throw errors


In [6]:

poe_bot.refreshAll()

In [6]:
ninja_prices = coordinator.getNinjaPrices()
if ninja_prices is None:
  print(f'cannot do stash.priceReport cos coordinator doesnt give prices')
  raise 404
div_price = ninja_prices["Divine Orb"]['chaosValue']

In [7]:
div_price

137.97

In [ ]:
stash.temp.reset()
stash.updateStashTemp()

stash.updateStashTemp call 1702312281.5165741
no items getOpenedStashInfo
updateTabInfo for tab with index 5
updating stash tab with index 5
no items getOpenedStashInfo
updateTabInfo for tab with index 5
updating stash tab with index 5
no items getOpenedStashInfo
updateTabInfo for tab with index 1
updating stash tab with index 1
no items getOpenedStashInfo
updateTabInfo for tab with index 1
updating stash tab with index 1
no items getOpenedStashInfo
updateTabInfo for tab with index 0
updating stash tab with index 0
no items getOpenedStashInfo
updateTabInfo for tab with index 0
updating stash tab with index 0
updateTabInfo for tab with index 2
updating stash tab with index 2
updateTabInfo for tab with index 2
updating stash tab with index 2
no items getOpenedStashInfo
updateTabInfo for tab with index 6
updating stash tab with index 6
current stash tab is fragment tab
no items getOpenedStashInfo
updateTabInfo for tab with index 6
updating stash tab with index 6
no items getOpenedStashInf

In [9]:
all_items = stash.getAllItems()

In [ ]:
essences = list(filter(lambda i: "Essence of" in i.name, all_items))
total_price = 0
high_ess_dict = {

}

for category_key in ["Deafening", "Shrieking"]:
  all_items_by_key_name = {}
  category_price = 0
  for item in essences:
    if not category_key in item.name:
      continue
    if all_items_by_key_name.get(item.name, None) is None:
      all_items_by_key_name[item.name] = 0

    all_items_by_key_name[item.name] += item.items_in_stack

  for item_key in all_items_by_key_name.keys():
    count = all_items_by_key_name[item_key]

    ninja_data = ninja_prices[item_key]
    price_per_item = ninja_data['chaosValue']
    total_price_for_stacks = count * price_per_item
    total_price += total_price_for_stacks
    category_price += total_price_for_stacks
    print(f"x{count:<4}| {price_per_item:<4}c | -{int(total_price_for_stacks)}c | {item_key}")
  print(f'{category_key}: {category_price}c')


high_ess_keys = ["Essence of Horror", "Essence of Delirium", "Essence of Hysteria","Essence of Insanity"]
for essence_item in essences:
  if essence_item.name in high_ess_keys:
    key_in_dict = high_ess_dict.get(essence_item.name, None)
    if key_in_dict is None:
      high_ess_dict[essence_item.name] = 0
    high_ess_dict[essence_item.name] += essence_item.items_in_stack

for item_key in high_ess_dict.keys():
  count = high_ess_dict[item_key]
  ninja_data = ninja_prices[item_key]
  price_per_item = ninja_data['chaosValue']
  total_price_for_stacks = count * price_per_item
  total_price += total_price_for_stacks
  category_price += total_price_for_stacks
  print(f"x{count:<4}| {price_per_item:<4}c | -{int(total_price_for_stacks)}c | {item_key}")

print(f'all essences: {total_price}c')
print(f'div_price {div_price}')
for price_multiplier in [1.1,1,0.95,0.9,0.85,0.8,0.75]:
  multiplied_price = total_price * price_multiplier
  price_in_div = int(multiplied_price // div_price)
  left_in_chaos = int(multiplied_price - price_in_div * div_price)
  multiplied_category_price = f'{price_multiplier*100}%: {price_in_div}d + {left_in_chaos} | {int(multiplied_price)}c'
  print(multiplied_category_price)

scarabs = list(filter(lambda i: "Scarab" in i.name, all_items))
total_price = 0
scarabs_dict = {

}
for item in scarabs:
  key_in_dict = scarabs_dict.get(item.name, None)
  if key_in_dict is None:
    scarabs_dict[item.name] = 0
  scarabs_dict[item.name] += item.items_in_stack

for item_key in scarabs_dict.keys():
  count = scarabs_dict[item_key]
  ninja_data = ninja_prices[item_key]
  price_per_item = ninja_data['chaosValue']
  total_price_for_stacks = count * price_per_item
  total_price += total_price_for_stacks
  print(f"x{count:<4}| {price_per_item:<4}c | -{int(total_price_for_stacks):<4}c | {item_key}")

print(f'total scarab price {total_price}c {total_price/div_price}d')
print(f'div_price {div_price}')
for price_multiplier in [0.9, 0.85, 0.8, 0.75, 0.7]:
  multiplied_price = total_price * price_multiplier
  price_in_div = int(multiplied_price // div_price)
  left_in_chaos = int(multiplied_price - price_in_div * div_price)
  multiplied_category_price = f'{price_multiplier*100}%: {price_in_div}d + {left_in_chaos} | {int(multiplied_price)}c'
  print(multiplied_category_price)

Deafening: 0c
Shrieking: 0c
all essences: 0c
div_price 137.97
110.00000000000001%: 0d + 0 | 0c
100%: 0d + 0 | 0c
95.0%: 0d + 0 | 0c
90.0%: 0d + 0 | 0c
85.0%: 0d + 0 | 0c
80.0%: 0d + 0 | 0c
75.0%: 0d + 0 | 0c


In [ ]:
poe_bot.ui.closeAll()

In [ ]:
raise "done"

In [ ]:
for item in all_items:
  print(item.raw)

{'l': ['RGRGGB'], 'Name': 'Golden Plate', 'a': 'Art/2DItems/Armours/BodyArmours/BodyStr1B.dds', 'unique_name': 'Gloom Suit', 'rarity': 'Rare', 'Class': None, 'ItemType': None, 'm_t': 0, 'c': 1, 'i': 1, 'RenderArt': 'Metadata/Items/Armours/BodyArmours/BodyStr13', 'items_in_stack': 0, 'item_mods': [], 'imr': ['AttackerTakesDamage1', 'LightningResist6', 'IncreasedLife9', 'LocalBaseArmourAndLife3', 'FireResist4', 'ColdResist1'], 'g': None, 's': None, 'ti': 0, 'LocationTopLeft': {'X': 10, 'Y': 0}, 'LocationBottomRight': {'X': 12, 'Y': 3}, 'TopLeft': {'X': 386, 'Y': 90}, 'BottomRight': {'X': 461, 'Y': 203}, 'tab_index': 0}
{'l': None, 'Name': 'The Fortunate', 'a': 'Art/2DItems/Divination/InventoryIcon.dds', 'unique_name': None, 'rarity': None, 'Class': None, 'ItemType': None, 'm_t': 0, 'c': 0, 'i': 0, 'RenderArt': 'Metadata/Items/DivinationCards/DivinationCardTheFortunate', 'items_in_stack': 5, 'item_mods': [], 'imr': [], 'g': None, 's': None, 'ti': 0, 'LocationTopLeft': {'X': 2, 'Y': 4}, 'L

In [ ]:
essences = list(filter(lambda i: "Scarab" in i.name, all_items))
total_price = 0
high_ess_dict = {

}

for category_key in ["Deafening", "Shrieking"]:
  all_items_by_key_name = {}
  category_price = 0
  for item in essences:
    if not category_key in item.name:
      continue
    if all_items_by_key_name.get(item.name, None) is None:
      all_items_by_key_name[item.name] = 0

    all_items_by_key_name[item.name] += item.items_in_stack

  for item_key in all_items_by_key_name.keys():
    count = all_items_by_key_name[item_key]

    ninja_data = ninja_prices[item_key]
    price_per_item = ninja_data['chaosValue']
    total_price_for_stacks = count * price_per_item
    total_price += total_price_for_stacks
    category_price += total_price_for_stacks
    print(f"x{count:<4}| {price_per_item:<4}c | -{int(total_price_for_stacks)}c | {item_key}")
  print(f'{category_key}: {category_price}c')


high_ess_keys = ["Essence of Horror", "Essence of Delirium", "Essence of Hysteria","Essence of Insanity"]
for essence_item in essences:
  if essence_item.name in high_ess_keys:
    key_in_dict = high_ess_dict.get(essence_item.name, None)
    if key_in_dict is None:
      high_ess_dict[essence_item.name] = 0
    high_ess_dict[essence_item.name] += essence_item.items_in_stack

for item_key in high_ess_dict.keys():
  count = high_ess_dict[item_key]
  ninja_data = ninja_prices[item_key]
  price_per_item = ninja_data['chaosValue']
  total_price_for_stacks = count * price_per_item
  total_price += total_price_for_stacks
  category_price += total_price_for_stacks
  print(f"x{count:<4}| {price_per_item:<4}c | -{int(total_price_for_stacks)}c | {item_key}")

print(f'all essences: {total_price}c')
print(f'div_price {div_price}')
for price_multiplier in [1.1,1,0.95,0.9,0.85,0.8,0.75]:
  multiplied_price = total_price * price_multiplier
  price_in_div = int(multiplied_price // div_price)
  left_in_chaos = int(multiplied_price - price_in_div * div_price)
  multiplied_category_price = f'{price_multiplier*100}%: {price_in_div}d + {left_in_chaos} | {int(multiplied_price)}c'
  print(multiplied_category_price)

all essences: 0c
div_price 137.97
110.00000000000001%: 0d + 0 | 0c
100%: 0d + 0 | 0c
95.0%: 0d + 0 | 0c
90.0%: 0d + 0 | 0c
85.0%: 0d + 0 | 0c
80.0%: 0d + 0 | 0c
75.0%: 0d + 0 | 0c
